In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/xview
!pip install numpy tensorflow-gpu
!pip install -r requirements.txt

In [ ]:
#!pip list
#!pip uninstall cupy
!pip uninstall cupy-cuda101
!pip install 'cupy>=7.4.0,<8.0.0'

Uninstalling cupy-cuda101-7.4.0:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/cupy/*
    /usr/local/lib/python3.6/dist-packages/cupy_cuda101-7.4.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/cupyx/*
Proceed (y/n)? y
  Successfully uninstalled cupy-cuda101-7.4.0
     |████████████████████████████████| 3.7MB 2.7MB/s 
  Created wheel for cupy: filename=cupy-7.7.0-cp36-cp36m-linux_x86_64.whl size=33271926 sha256=244e925f1d5eb177a3629eb1a429a210e73422e95b05bc0af4c1fe0cc2e0621e
  Stored in directory: /root/.cache/pip/wheels/c9/f6/ef/9cb72172a8f7efe4f4a39aeb011202fc80377ec12b3903022d
Successfully built cupy


In [ ]:
%cd /content/drive/My Drive/xview/model/
%mkdir /content/sample_data/output
%mkdir /content/sample_data/output/process_images
%mkdir /content/sample_data/output/csv

/content/drive/My Drive/xview/model


In [ ]:
!python process_data.py --input_dir "/content/drive/My Drive/xview/xBD"  --output_dir "/content/sample_data/output/process_images" --output_dir_csv "/content/sample_data/output/csv" --val_split_pct 0.1

INFO:root:Started Processing for Data
100% 5598/5598 [2:36:22<00:00,  1.68s/it]
INFO:root:Finished Processing Data


In [ ]:
from PIL import Image
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import math
import random
import argparse
import logging
import json
import cv2
import datetime

from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight
import shapely.wkt
import shapely
from shapely.geometry import Polygon
from collections import defaultdict

import tensorflow as tf
import keras
import ast
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Add, Input, Concatenate
from keras.models import Model
from keras.applications.vgg19 import VGG19
from keras import backend as K

In [ ]:
def ordinal_loss(y_true, y_pred):
    weights = K.cast(K.abs(K.argmax(y_true, axis=1) - K.argmax(y_pred, axis=1))/(K.int_shape(y_pred)[1] - 1), dtype='float32')
    return (1.0 + weights) * keras.losses.categorical_crossentropy(y_true, y_pred )

In [ ]:
def generate_vgg19_model():
  weights = 'imagenet'
  inputs = Input(shape=(128, 128, 3))

  base_model = VGG19(include_top=False, weights=weights, input_shape=(128, 128, 3))

  for layer in base_model.layers:
    layer.trainable = False

  x = Conv2D(32, (5, 5), strides=(1, 1), padding='same', activation='relu', input_shape=(128, 128, 3))(inputs)
  x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

  x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
  x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

  x = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x)
  x = MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None)(x)

  x = Flatten()(x)

  base_resnet = base_model(inputs)
  base_resnet = Flatten()(base_resnet)

  concated_layers = Concatenate()([x, base_resnet])

  concated_layers = Dense(2024, activation='relu')(concated_layers)
  concated_layers = Dense(524, activation='relu')(concated_layers)
  concated_layers = Dense(124, activation='relu')(concated_layers)
  output = Dense(4, activation='relu')(concated_layers)

  model = Model(inputs=inputs, outputs=output)
  return model

In [ ]:
from PIL import Image
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import math
import random
import argparse
import logging
import json
import cv2
import datetime

from sklearn.metrics import f1_score
from sklearn.utils.class_weight import compute_class_weight
import shapely.wkt
import shapely
from shapely.geometry import Polygon
from collections import defaultdict

import tensorflow as tf
import keras
import ast
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Add, Input, Concatenate
from keras.models import Model
from keras.applications.vgg19 import VGG19
from keras import backend as K

In [ ]:
logging.basicConfig(level=logging.INFO)

# Configurations
NUM_WORKERS = 4 
NUM_CLASSES = 4
BATCH_SIZE = 64
NUM_EPOCHS = 40 
LEARNING_RATE = 0.0001
RANDOM_SEED = 123
LOG_STEP = 150
LOG_DIR = '/path/to/logs' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

damage_intensity_encoding = dict()
damage_intensity_encoding[3] = '3'
damage_intensity_encoding[2] = '2' 
damage_intensity_encoding[1] = '1' 
damage_intensity_encoding[0] = '0' 


In [ ]:
def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


In [ ]:
def validation_generator(test_csv, test_dir):
    df = pd.read_csv(test_csv)
    df = df.replace({"labels" : damage_intensity_encoding })

    gen = keras.preprocessing.image.ImageDataGenerator(
                             rescale=1/255.)


    return gen.flow_from_dataframe(dataframe=df,
                                   directory=test_dir,
                                   x_col='uuid',
                                   y_col='labels',
                                   batch_size=BATCH_SIZE,
                                   shuffle=False,
                                   seed=RANDOM_SEED,
                                   class_mode="categorical",
                                   target_size=(128, 128))


In [ ]:
def augment_data(df, in_dir):

    df = df.replace({"labels" : damage_intensity_encoding })
    gen = keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True,
                             vertical_flip=True,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             rescale=1/255.)
    return gen.flow_from_dataframe(dataframe=df,
                                   directory=in_dir,
                                   x_col='uuid',
                                   y_col='labels',
                                   batch_size=BATCH_SIZE,
                                   seed=RANDOM_SEED,
                                   class_mode="categorical",
                                   target_size=(128, 128))

In [ ]:
def train_model(train_data, train_csv, test_data, test_csv, model_in, model_out):

    model = generate_vgg19_model()

    # Add model weights if provided by user
    if model_in is not None:
        model.load_weights(model_in)

    df = pd.read_csv(train_csv)
    class_weights = compute_class_weight('balanced', np.unique(df['labels'].to_list()), df['labels'].to_list());
    d_class_weights = dict(enumerate(class_weights))

    samples = df['uuid'].count()
    steps = np.ceil(samples/BATCH_SIZE)

    # Augments the training data
    train_gen_flow = augment_data(df, train_data)

    #Set up tensorboard logging
    tensorboard_callbacks = keras.callbacks.TensorBoard(log_dir=LOG_DIR,
                                                        batch_size=BATCH_SIZE)

    
    #Filepath to save model weights
    filepath = model_out + "-saved-model-{epoch:02d}-{accuracy:.2f}.hdf5"
    checkpoints = keras.callbacks.ModelCheckpoint(filepath,
                                                    monitor='accuracy',
                                                    verbose=1,
                                                    mode='max',
                                                    save_best_only=True)

    #Adds adam optimizer
    adam = keras.optimizers.Adam(lr=LEARNING_RATE,
                                    beta_1=0.9,
                                    beta_2=0.999,
                                    decay=0.0,
                                    amsgrad=False)


    model.compile(loss=ordinal_loss, optimizer=adam, metrics=['accuracy', f1])

    #Training begins
    model.fit_generator(generator=train_gen_flow,
                        steps_per_epoch=steps,
                        epochs=NUM_EPOCHS,
                        workers=NUM_WORKERS,
                        use_multiprocessing=True,
                        class_weight=d_class_weights,
                        callbacks=[tensorboard_callbacks, checkpoints],
                        verbose=1)


    #Evalulate f1 weighted scores on validation set
    validation_gen = validation_generator(test_csv, test_data)
    predictions = model.predict(validation_gen)

    val_trues = validation_gen.classes
    val_pred = np.argmax(predictions, axis=-1)
	
    precision = precision(y_true, y_pred)
    print('precision: {}').format(precision)
    recall = recall(y_true, y_pred)
    print('recall: {}').format(recall)
    f1_weighted = 2*((precision*recall)/(precision+recall+K.epsilon()))
    
    print(f1_weighted)


In [ ]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-ca07a89c-5d1a-f1da-3f43-4d5b14ecf1a8)


In [ ]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision


    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
train_model("/content/sample_data/output/process_images", "/content/sample_data/output/csv/train.csv", "/content/sample_data/output/process_images", "/content/sample_data/output/csv/test.csv", None, "/content/drive/My Drive/xview/baseline_model_vgg19/")

80142336/80134624 [==============================] - 2s 0us/step
Found 146508 validated image filenames belonging to 4 classes.


Instructions for updating:
Please use Model.fit, which supports generators.


Instructions for updating:
Please use Model.fit, which supports generators.


Epoch 1/40


   1/2290 [..............................] - ETA: 18s - loss: 8.2098 - accuracy: 0.0625 - f1: 0.1136WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


...

2289/2290 [============================>.] - ETA: 0s - loss: 6.1762 - accuracy: 0.6984 - f1: 0.4820WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.


2290/2290 [==============================] - ETA: 0s - loss: 6.1753 - accuracy: 0.6984 - f1: 0.4820
Epoch 00030: accuracy improved from 0.69228 to 0.69841, saving model to /content/drive/My Drive/xview/baseline_model_vgg19/-saved-model-30-0.70.hdf5
2290/2290 [==============================] - 785s 343ms/step - loss: 6.1753 - accuracy: 0.6984 - f1: 0.4820
Epoch 31/40


2289/2290 [============================>.] - ETA: 0s - loss: 6.1445 - accuracy: 0.7098 - f1: 0.4864WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.


2290/2290 [==============================] - ETA: 0s - loss: 6.1447 - accuracy: 0.7098 - f1: 0.4864
Epoch 00031: accuracy improved from 0.69841 to 0.70977, saving model to /content/drive/My Drive/xview/baseline_model_vgg19/-saved-model-31-0.71.hdf5
2290/2290 [==============================] - 786s 343ms/step - loss: 6.1447 - accuracy: 0.7098 - f1: 0.4864
Epoch 32/40


 484/2290 [=====>........................] - ETA: 10:16 - loss: 6.1905 - accuracy: 0.7128 - f1: 0.4878